In [1]:
import mmap
import time
from time import strftime, localtime

import import_ipynb
from library import *
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical
from torch_geometric.nn import NNConv, global_mean_pool, GraphUNet, TopKPooling, GCNConv
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from torch.utils.tensorboard import SummaryWriter
from sys import getsizeof

import glob
import os
import random
import json
import sys
import shutil


torch.set_printoptions(threshold=10_000)
np.set_printoptions(threshold=sys.maxsize)

os.chdir("C:/Users/user/Desktop/suhwan/connection_test/python_agent/shallow")

folderList = glob.glob("history*")

pathName = "history" + str(len(folderList))

print(pathName)


os.mkdir(pathName)



writer = SummaryWriter(pathName)
%matplotlib inline


BUFFER_SIZE = 200000

modelNum = 0
availableJobNum = 0
nodeNum = 0
jobWaitingLength = 0
adjacency = 0

importing Jupyter notebook from library.ipynb
importing Jupyter notebook from job.ipynb
importing Jupyter notebook from jobqueue.ipynb
importing Jupyter notebook from network.ipynb
importing Jupyter notebook from dataplane.ipynb


C:\Users\user\anaconda3\envs\omnetTest\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


history0


In [2]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

print(torch.cuda.is_available())

print(torch.cuda.get_device_name(device = 0))

device = torch.device('cuda:0')
print(device)

print(type(device))

kkkkk = torch.Tensor(1)

kkkkk = kkkkk.cuda()

print(kkkkk.cuda)
print(type(kkkkk.device))
print(kkkkk.device == device)


True
NVIDIA GeForce RTX 3050
cuda:0
<class 'torch.device'>
<built-in method cuda of Tensor object at 0x0000026A81BC2C50>
<class 'torch.device'>
True


In [3]:
learning_rate   = 0.0003
gamma           = 0.95
entropy_weight  = 0.001
val_loss_coef = 0.8
'''
entropy_weight : loss함수에 entropy라는 걸 더해주는 테크닉에서의 weight입니다.
이 기법은 A3C 논문에 나와있으며, 쓰는 이유는 exploration을 촉진하기 위해서입니다.
각 액션을 취할 확률을 거의 균등하게끔 업데이트해줌으로써 다양한 액션을 해볼 기회가 많아집니다.
actor-critic에서 이 기법으로 인한 성능 개선이 매우 효과적인 것으로 알고있으므로, 적용해주시면 좋을 듯 합니다.
'''
lmbda         = 0.8
eps_clip      = 0.2
'''
위 두 인자는 PPO에서 씁니다.
'''

batch_size = 128

loss_coef = 0.5

node_feature_num = 100
queue_feature_num = 100
hidden_feature_num = 0

job_generate_rate = 0.003

train_cycle = 300
is_train = True
train_quitient = 0

replay_buffer_size = 10000
history_learning_time = 6

if not is_train:
    train_quitient = train_cycle
    

# 통신 관련 초기화

In [4]:
# import os.path
# import os
# FIFO_FILENAME = './fifo-test'

# if not os.path.exists(FIFO_FILENAME):
#     os.mkfifo(FIFO_FILENAME)
#     if os.path.exists(FIFO_FILENAME):
#         fp_fifo = open(FIFO_FILENAME, "rw")

# for i in range(128):
#     fp_fifo.write("Hello,MakeCode\n")
#     fp_fifo.write("")

import sys 
import win32pipe, win32file, pywintypes

PIPE_NAME = "\\\\.\\pipe\\omnetPipe2"
BUFFER_SIZE = 200000

try:
    pipe = win32pipe.CreateNamedPipe(
        PIPE_NAME,
        win32pipe.PIPE_ACCESS_DUPLEX,
        win32pipe.PIPE_TYPE_MESSAGE | win32pipe.PIPE_READMODE_MESSAGE | win32pipe.PIPE_WAIT,
        1,
        BUFFER_SIZE,
        BUFFER_SIZE,
        0,
        None
    )    
except:
    pass

win32pipe.ConnectNamedPipe(pipe, None)



0

# 통신 관련 함수

In [5]:
def sendOmnetMessage(msg):
    win32file.WriteFile(pipe, msg.encode('utf-8'))
    
def getOmnetMessage():
    response_byte = win32file.ReadFile(pipe, BUFFER_SIZE)
    response_str = response_byte[1].decode('utf-8')
    return response_str

def closePipe():
    win32file.CloseHandle(pipe)

In [6]:
initial_message = getOmnetMessage()

networkInfo = json.loads(initial_message)

modelNum = int(networkInfo['modelNum'])
availableJobNum = int(networkInfo['availableJobNum'])
nodeNum = int(networkInfo['nodeNum'])
jobWaitingLength = int(networkInfo['jobWaitingQueueLength'])
adjacency = eval(networkInfo['adjacencyList'])
episode_length = int(networkInfo['episode_length'])

node_feature_num = 2 * (modelNum * availableJobNum)
queue_feature_num = (nodeNum + modelNum) * jobWaitingLength
hidden_feature_num = 10*(node_feature_num + queue_feature_num)

sendOmnetMessage("init") # 입력 끝나면 omnet에 전송

print("네트워크 초기화 완료")

print(f"노드 개수 : {nodeNum}")
print(f"네트워크 최대 job 개수 : {availableJobNum}")
print(f"job 대기 가능 개수 : {jobWaitingLength}")
print(f"최대 subtask 개수 : {modelNum}")
print(f"인접 리스트 : {adjacency}")
print(f"node_feature_num : {node_feature_num}")
print(f"queue_feature_num : {queue_feature_num}")
print(f"episode_length : {episode_length}")

네트워크 초기화 완료
노드 개수 : 7
네트워크 최대 job 개수 : 20
job 대기 가능 개수 : 15
최대 subtask 개수 : 5
인접 리스트 : [[0, 3, 1, 3, 1, 4, 1, 5, 2, 3, 2, 4, 2, 5, 3, 4, 3, 6, 4, 5, 4, 6, 5, 6], [3, 0, 3, 1, 4, 1, 5, 1, 3, 2, 4, 2, 5, 2, 4, 3, 6, 3, 5, 4, 6, 4, 6, 5]]
node_feature_num : 200
queue_feature_num : 180
episode_length : 100


In [7]:
class actor_network(nn.Module):
    def __init__(self):
        super(actor_network, self).__init__()
        self.data = []
        self.history = []

        self.x_mean = 0
        self.x2_mean = 0
        self.sd = 0
        self.reward_sample_num = 0

        self.step = 0

        
        # print(f"self.adjacency : ", self.adjacency.shape)

        self.pi_mlp1 = nn.Sequential(nn.Linear(1, node_feature_num * int(node_feature_num//2)), nn.ReLU())
        self.pi_s_ecc1 = NNConv(node_feature_num, int(node_feature_num//2), self.pi_mlp1, aggr='mean')

        self.pi_mlp2 = nn.Sequential(nn.Linear(1, int(node_feature_num//2) * int(node_feature_num//2)), nn.ReLU())
        self.pi_s_ecc2 = NNConv(int(node_feature_num//2), int(node_feature_num//2), self.pi_mlp2, aggr='mean')

        self.pi_graph_u_net1 = GraphUNet(int(node_feature_num//2), 50, int(node_feature_num//2), 3, 0.8)
        self.pi_graph_u_net2 = GraphUNet(int(node_feature_num//2), 50, int(node_feature_num//2), 3, 0.8)

        self.pi_backbone = nn.Sequential(
            nn.Linear((int(node_feature_num//2) + queue_feature_num) * 2, hidden_feature_num),
            nn.ReLU(),
            nn.Linear(hidden_feature_num, hidden_feature_num),
            nn.ReLU(),
        )

        # prob_fc : 각 액션에 대한 확률.
        self.pi_prob_fc = nn.Linear(hidden_feature_num, nodeNum + 1) # nodeNum + voidAction


        # prob_fc : 각 액션에 대한 확률.
        self.v_value_fc = nn.Linear(hidden_feature_num, 1)
        
        self.optimizer = optim.Adam(self.parameters(), lr=learning_rate)
              
        
    # policy DNN
    def pi(self, state):
        data, job_waiting_feature = state
        node_feature_1, link_feature_1, adjacency_1 = data[0].x, data[0].edge_attr, data[0].edge_index
        node_feature_2, link_feature_2, adjacency_2 = data[1].x, data[1].edge_attr, data[1].edge_index

        job_waiting_feature_1 = job_waiting_feature[0]
        job_waiting_feature_2 = job_waiting_feature[1]

        if job_waiting_feature_2.device == 'cuda':
            link_feature_1 = link_feature_1.cuda()
            adjacency_1 = adjacency_1.cuda()

            link_feature_2 = link_feature_2.cuda()
            adjacency_2 = adjacency_2.cuda()

        
        # =========================================================================
        node_feature_1 = F.relu(self.pi_s_ecc1(node_feature_1, adjacency_1, link_feature_1))
        node_feature_1 = F.relu(self.pi_graph_u_net1(node_feature_1, adjacency_1))

        # node_feature_1 = F.relu(self.pi_s_ecc2(node_feature_1, adjacency_1, link_feature_1))
        # node_feature_1 = F.relu(self.pi_graph_u_net2(node_feature_1, adjacency_1))

        data_num_1 = len(node_feature_1) // nodeNum
        
        readout_1 = global_mean_pool(node_feature_1, data[0].batch)

        concat_1 = torch.cat([readout_1, job_waiting_feature_1], dim=1) # 여기에 job waiting 벡터 붙이기.
        # =========================================================================
        node_feature_2 = F.relu(self.pi_s_ecc1(node_feature_2, adjacency_2, link_feature_2))
        node_feature_2 = F.relu(self.pi_graph_u_net1(node_feature_2, adjacency_2))

        # node_feature_2 = F.relu(self.pi_s_ecc2(node_feature_2, adjacency_2, link_feature_2))
        # node_feature_2 = F.relu(self.pi_graph_u_net2(node_feature_2, adjacency_2))
        
        data_num_2 = len(node_feature_2) // nodeNum
        
        readout_2 = global_mean_pool(node_feature_2, data[0].batch)

        concat_2 = torch.cat([readout_2, job_waiting_feature_2], dim=1) # 여기에 job waiting 벡터 붙이기.

        # =========================================================================

        concat = torch.cat([concat_1, concat_2], dim=1)

        feature_extract = self.pi_backbone(concat)

        # print(feature_extract)

        output = self.pi_prob_fc(feature_extract) 

        prob = F.softmax(output, dim=1)
        
        # 아래는 엔트로피 구하는 과정
        log_prob = F.log_softmax(output, dim=1)
        entropy = - (log_prob * prob).sum(1, keepdim=True)
        
        return prob, entropy, output

    # advantage network
    def v(self, state):
        data, job_waiting_feature = state
        node_feature_1, link_feature_1, adjacency_1 = data[0].x, data[0].edge_attr, data[0].edge_index
        node_feature_2, link_feature_2, adjacency_2 = data[1].x, data[1].edge_attr, data[1].edge_index

        job_waiting_feature_1 = job_waiting_feature[0]
        job_waiting_feature_2 = job_waiting_feature[1]

        if job_waiting_feature_2.device == 'cuda':
            link_feature_1 = link_feature_1.cuda()
            adjacency_1 = adjacency_1.cuda()

            link_feature_2 = link_feature_2.cuda()
            adjacency_2 = adjacency_2.cuda()

        
        # =========================================================================
        node_feature_1 = F.relu(self.pi_s_ecc1(node_feature_1, adjacency_1, link_feature_1))
        node_feature_1 = F.relu(self.pi_graph_u_net1(node_feature_1, adjacency_1))

        # node_feature_1 = F.relu(self.pi_s_ecc2(node_feature_1, adjacency_1, link_feature_1))
        # node_feature_1 = F.relu(self.pi_graph_u_net2(node_feature_1, adjacency_1))

        data_num_1 = len(node_feature_1) // nodeNum
        
        readout_1 = global_mean_pool(node_feature_1, data[0].batch)

        concat_1 = torch.cat([readout_1, job_waiting_feature_1], dim=1) # 여기에 job waiting 벡터 붙이기.

        # =========================================================================
        node_feature_2 = F.relu(self.pi_s_ecc1(node_feature_2, adjacency_2, link_feature_2))
        node_feature_2 = F.relu(self.pi_graph_u_net1(node_feature_2, adjacency_2))

        # node_feature_2 = F.relu(self.pi_s_ecc2(node_feature_2, adjacency_2, link_feature_2))
        # node_feature_2 = F.relu(self.pi_graph_u_net2(node_feature_2, adjacency_2))

        data_num_2 = len(node_feature_2) // nodeNum
        
        readout_2 = global_mean_pool(node_feature_2, data[0].batch)

        concat_2 = torch.cat([readout_2, job_waiting_feature_2], dim=1) # 여기에 job waiting 벡터 붙이기.

        # =========================================================================

        concat = torch.cat([concat_1, concat_2], dim=1)

        feature_extract = self.pi_backbone(concat)

        
        value = self.v_value_fc(feature_extract) # 앞부분은 pi랑 공유해야 하고, concat -> value_fc를 거치는 것만 다름.
        return value
        
    def put_data(self, transition):
        self.data.append(transition)

    def set_reward(self, reward):
        self.data[-1][3] = reward

    def set_reward(self, reward):
        self.data[-1][3] = reward
        
    def return_link_dict(sel, ad):
        result = {}
        for i in range(len(ad[0])//2):
            result[f'{ad[0][2*i]}{ad[0][2 *(i)+1]}'] = i
            
        return result

    def return_new_mean(self, mean, num, new_data):
        result = (mean * num + new_data) / (num + 1)
        return result
        
    def return_new_sd(self, square_mean, mean):
        result = (square_mean - mean**2)**0.5
        
        return result
        
    def return_normalize_reward(self, reward):
        self.x_mean = self.return_new_mean(self.x_mean, self.reward_sample_num, reward)
        self.x2_mean = self.return_new_mean(self.x2_mean, self.reward_sample_num, reward**2)
        self.sd = self.return_new_sd(self.x2_mean, self.x_mean)
        
        if self.sd == 0:
                z_normalized = 0
        else:  
            z_normalized = (reward - self.x_mean) / self.sd

        self.reward_sample_num += 1
        
        return z_normalized
        
    
    # make_batch, train_net은 맨 위에 코드 기반 링크와 거의 동일합니다.

    def make_batch_history(self):
        sample_index = torch.randperm(len(self.history))[:batch_size]

        sampled_data = []
        for sample_idx in sample_index:
            sampled_data.append(self.history[sample_idx])

        network_1_lst, network_2_lst = [], []
        next_network_1_lst, next_network_2_lst = [], []

        job_waiting_1_lst, job_waiting_2_lst = [], []
        next_job_waiting_1_lst, next_job_waiting_2_lst = [], []

        a_lst, r_lst, prob_a_lst, sojourn_time_lst, action_mask_lst = [], [], [], [], []
        
        entropy_lst = []

        for idx, transition in enumerate(sampled_data):
            #self.history.append(transition)
            network, job_waiting, a, r, nxt_network, nxt_job_waiting, prob_a, entropy, sojourn_time, action_mask = transition

            r_lst.append([r/10.0])

            network_1_lst.append(network[0])
            network_2_lst.append(network[1])

            job_waiting_1_lst.append(job_waiting[0].tolist())
            job_waiting_2_lst.append(job_waiting[1].tolist())

            a_lst.append([a])

            next_network_1_lst.append(nxt_network[0])
            next_network_2_lst.append(nxt_network[1])
            
            next_job_waiting_1_lst.append(nxt_job_waiting[0].tolist())
            next_job_waiting_2_lst.append(nxt_job_waiting[1].tolist())

            prob_a_lst.append([prob_a])
            entropy_lst.append([entropy])
            sojourn_time_lst.append([sojourn_time * 5])
            action_mask_lst.append(action_mask)

            if idx == batch_size - 1:
                self.data = self.data[batch_size:]
                break

        else:
            self.data = []
        
        # r_lst = np.array(r_lst)
        # r_lst = (r_lst - np.mean(r_lst)) / np.std(r_lst)
        # r_lst = r_lst.tolist()
        

        # gnn sample을 배치단위로 inference하려면 이렇게 묶어줘야 함.
        network_1_loader = DataLoader(network_1_lst, batch_size=len(network_1_lst))
        next_network_1_loader = DataLoader(next_network_1_lst, batch_size=len(network_1_lst))
        network_1_batch = next(iter(network_1_loader))
        next_network_1_batch = next(iter(next_network_1_loader))

        network_2_loader = DataLoader(network_2_lst, batch_size=len(network_2_lst))
        next_network_2_loader = DataLoader(next_network_2_lst, batch_size=len(network_2_lst))
        network_2_batch = next(iter(network_2_loader))
        next_network_2_batch = next(iter(next_network_2_loader))

        # print(job_waiting_lst)
        
        # job_waiting = torch.tensor(np.array(job_waiting_lst), dtype=torch.float)
        job_waiting_1 = torch.squeeze(torch.tensor(np.array(job_waiting_1_lst), dtype=torch.float), dim=1)
        job_waiting_2 = torch.squeeze(torch.tensor(np.array(job_waiting_2_lst), dtype=torch.float), dim=1)

        a = torch.tensor(a_lst)
        r = torch.tensor(r_lst, dtype=torch.float)
        sojourn_time = torch.Tensor(sojourn_time_lst)

        next_job_waiting_1 = torch.squeeze(torch.tensor(np.array(next_job_waiting_1_lst), dtype=torch.float), dim=1)
        next_job_waiting_2 = torch.squeeze(torch.tensor(np.array(next_job_waiting_2_lst), dtype=torch.float), dim=1)
        
        prob_a = torch.tensor(prob_a_lst, dtype=torch.float)
        entropy = torch.tensor(entropy_lst, dtype=torch.float)
        action_mask_lst = torch.tensor(action_mask_lst)
        
        # self.data = []
        return [network_1_batch, network_2_batch], [job_waiting_1, job_waiting_2], a, r, [next_network_1_batch, next_network_2_batch], [next_job_waiting_1, next_job_waiting_2], prob_a, entropy, sojourn_time, action_mask_lst

        
    def make_batch(self):
        network_1_lst, network_2_lst = [], []
        next_network_1_lst, next_network_2_lst = [], []

        job_waiting_1_lst, job_waiting_2_lst = [], []
        next_job_waiting_1_lst, next_job_waiting_2_lst = [], []

        a_lst, r_lst, prob_a_lst, sojourn_time_lst, action_mask_lst = [], [], [], [], []
        
        entropy_lst = []

        for idx, transition in enumerate(self.data):
            #self.history.append(transition)
            network, job_waiting, a, r, nxt_network, nxt_job_waiting, prob_a, entropy, sojourn_time, action_mask = transition

            r_lst.append([r/10.0])

            network_1_lst.append(network[0])
            network_2_lst.append(network[1])

            job_waiting_1_lst.append(job_waiting[0].tolist())
            job_waiting_2_lst.append(job_waiting[1].tolist())

            a_lst.append([a])

            next_network_1_lst.append(nxt_network[0])
            next_network_2_lst.append(nxt_network[1])
            
            next_job_waiting_1_lst.append(nxt_job_waiting[0].tolist())
            next_job_waiting_2_lst.append(nxt_job_waiting[1].tolist())

            prob_a_lst.append([prob_a])
            entropy_lst.append([entropy])
            sojourn_time_lst.append([sojourn_time * 5])
            action_mask_lst.append(action_mask)

            if idx == batch_size - 1:
                self.data = self.data[batch_size:]
                break

        else:
            self.data = []
        
        # r_lst = np.array(r_lst)
        # r_lst = (r_lst - np.mean(r_lst)) / np.std(r_lst)
        # r_lst = r_lst.tolist()
        

        # gnn sample을 배치단위로 inference하려면 이렇게 묶어줘야 함.
        network_1_loader = DataLoader(network_1_lst, batch_size=len(network_1_lst))
        next_network_1_loader = DataLoader(next_network_1_lst, batch_size=len(network_1_lst))
        network_1_batch = next(iter(network_1_loader))
        next_network_1_batch = next(iter(next_network_1_loader))

        network_2_loader = DataLoader(network_2_lst, batch_size=len(network_2_lst))
        next_network_2_loader = DataLoader(next_network_2_lst, batch_size=len(network_2_lst))
        network_2_batch = next(iter(network_2_loader))
        next_network_2_batch = next(iter(next_network_2_loader))

        # print(job_waiting_lst)
        
        # job_waiting = torch.tensor(np.array(job_waiting_lst), dtype=torch.float)
        job_waiting_1 = torch.squeeze(torch.tensor(np.array(job_waiting_1_lst), dtype=torch.float), dim=1)
        job_waiting_2 = torch.squeeze(torch.tensor(np.array(job_waiting_2_lst), dtype=torch.float), dim=1)

        a = torch.tensor(a_lst)
        r = torch.tensor(r_lst, dtype=torch.float)
        sojourn_time = torch.Tensor(sojourn_time_lst)

        next_job_waiting_1 = torch.squeeze(torch.tensor(np.array(next_job_waiting_1_lst), dtype=torch.float), dim=1)
        next_job_waiting_2 = torch.squeeze(torch.tensor(np.array(next_job_waiting_2_lst), dtype=torch.float), dim=1)
        
        prob_a = torch.tensor(prob_a_lst, dtype=torch.float)
        entropy = torch.tensor(entropy_lst, dtype=torch.float)
        action_mask_lst = torch.tensor(action_mask_lst)
        
        # self.data = []
        return [network_1_batch, network_2_batch], [job_waiting_1, job_waiting_2], a, r, [next_network_1_batch, next_network_2_batch], [next_job_waiting_1, next_job_waiting_2], prob_a, entropy, sojourn_time, action_mask_lst
    
    
    def make_first_batch(self):
        self.data = self.data[::-1]
        
        network_1_lst, network_2_lst = [], []
        next_network_1_lst, next_network_2_lst = [], []

        job_waiting_1_lst, job_waiting_2_lst = [], []
        next_job_waiting_1_lst, next_job_waiting_2_lst = [], []

        a_lst, r_lst, prob_a_lst, sojourn_time_lst, action_mask_lst = [], [], [], [], []
        
        entropy_lst = []

        for idx, transition in enumerate(self.data):
            #self.history.append(transition)
            network, job_waiting, a, r, nxt_network, nxt_job_waiting, prob_a, entropy, sojourn_time, action_mask = transition

            r_lst.append([r/10.0])

            network_1_lst.append(network[0])
            network_2_lst.append(network[1])

            job_waiting_1_lst.append(job_waiting[0].tolist())
            job_waiting_2_lst.append(job_waiting[1].tolist())

            a_lst.append([a])

            next_network_1_lst.append(nxt_network[0])
            next_network_2_lst.append(nxt_network[1])
            
            next_job_waiting_1_lst.append(nxt_job_waiting[0].tolist())
            next_job_waiting_2_lst.append(nxt_job_waiting[1].tolist())

            prob_a_lst.append([prob_a])
            entropy_lst.append([entropy])
            sojourn_time_lst.append([sojourn_time * 5])
            action_mask_lst.append(action_mask)

            if idx == batch_size - 1:
                self.data = self.data[batch_size:]
                break

        else:
            self.data = []
        
        # r_lst = np.array(r_lst)
        # r_lst = (r_lst - np.mean(r_lst)) / np.std(r_lst)
        # r_lst = r_lst.tolist()
        

        # gnn sample을 배치단위로 inference하려면 이렇게 묶어줘야 함.
        network_1_loader = DataLoader(network_1_lst, batch_size=len(network_1_lst))
        next_network_1_loader = DataLoader(next_network_1_lst, batch_size=len(network_1_lst))
        network_1_batch = next(iter(network_1_loader))
        next_network_1_batch = next(iter(next_network_1_loader))

        network_2_loader = DataLoader(network_2_lst, batch_size=len(network_2_lst))
        next_network_2_loader = DataLoader(next_network_2_lst, batch_size=len(network_2_lst))
        network_2_batch = next(iter(network_2_loader))
        next_network_2_batch = next(iter(next_network_2_loader))

        # print(job_waiting_lst)
        
        # job_waiting = torch.tensor(np.array(job_waiting_lst), dtype=torch.float)
        job_waiting_1 = torch.squeeze(torch.tensor(np.array(job_waiting_1_lst), dtype=torch.float), dim=1)
        job_waiting_2 = torch.squeeze(torch.tensor(np.array(job_waiting_2_lst), dtype=torch.float), dim=1)

        a = torch.tensor(a_lst)
        r = torch.tensor(r_lst, dtype=torch.float)
        sojourn_time = torch.Tensor(sojourn_time_lst)

        next_job_waiting_1 = torch.squeeze(torch.tensor(np.array(next_job_waiting_1_lst), dtype=torch.float), dim=1)
        next_job_waiting_2 = torch.squeeze(torch.tensor(np.array(next_job_waiting_2_lst), dtype=torch.float), dim=1)
        
        prob_a = torch.tensor(prob_a_lst, dtype=torch.float)
        entropy = torch.tensor(entropy_lst, dtype=torch.float)
        action_mask_lst = torch.tensor(action_mask_lst)
        
        # self.data = []
        return [network_1_batch, network_2_batch], [job_waiting_1, job_waiting_2], a, r, [next_network_1_batch, next_network_2_batch], [next_job_waiting_1, next_job_waiting_2], prob_a, entropy, sojourn_time, action_mask_lst
    
    
    
    def train_net(self):
        self = self.cuda()
        first = True
        pre_advantage = 0.0
        while len(self.data) > 0:
            torch.cuda.empty_cache()
            if first:
                network_batch, job_waiting, a, r, next_network_batch, next_job_waiting, prob_a, entropy, sojourn_time, action_mask = self.make_first_batch()
                first = False
            else:
                network_batch, job_waiting, a, r, next_network_batch, next_job_waiting, prob_a, entropy, sojourn_time, action_mask = self.make_batch()
            
            network_batch_1 = network_batch[0].clone().cuda()
            network_batch_2 = network_batch[1].clone().cuda()

            network_batch = [network_batch_1, network_batch_2]

            job_waiting_1 = job_waiting[0].clone().cuda()
            job_waiting_2 = job_waiting[1].clone().cuda()

            job_waiting = [job_waiting_1, job_waiting_2]

            a = a.clone().cuda()
            r = r.clone().cuda()

            next_network_batch_1 = next_network_batch[0].clone().cuda()
            next_network_batch_2 = next_network_batch[1].clone().cuda()

            next_network_batch = [next_network_batch_1, next_network_batch_2]

            next_job_waiting_1 = next_job_waiting[0].clone().cuda()
            next_job_waiting_2 = next_job_waiting[1].clone().cuda()

            next_job_waiting = [next_job_waiting_1, next_job_waiting_2]

            prob_a = prob_a.clone().cuda()
            entropy = entropy.clone().cuda()
            sojourn_time = sojourn_time.clone().cuda()
            gamma_gpu = torch.Tensor([gamma]).clone().cuda()
            action_mask = action_mask.clone().cuda()

            for i in range(2):
                
                td_target = r + (gamma_gpu**sojourn_time) * self.v([next_network_batch, next_job_waiting])
                
                delta = td_target - self.v([network_batch, job_waiting])
                delta = delta.detach().to('cpu').numpy()
                advantage_lst = []
                advantage = pre_advantage
                for i, delta_t in enumerate(delta):
                    advantage = (gamma_gpu**sojourn_time[i][0]) * lmbda * advantage + delta_t[0]
                    advantage_lst.append([advantage])
                # advantage_lst.reverse()
                advantage_lst = torch.tensor(advantage_lst, dtype=torch.float).cuda()

                pre_advantage = advantage

                v_loss = val_loss_coef * F.smooth_l1_loss(self.v([network_batch, job_waiting]) , td_target.detach())
                #print("v_loss", v_loss)
                
                self.optimizer.zero_grad()
                v_loss.mean().backward()
                self.optimizer.step()

                pi, current_entropy, outputs = self.pi([network_batch, job_waiting])

                # true가 valid action

                #outputs = outputs.clone().detach()


                outputs = outputs * action_mask

                exp_sum = torch.sum(torch.exp(outputs), dim=1) - torch.sum(action_mask.logical_not(), dim=1) # 0인 애들 빼줌
                exp_sum = exp_sum.view(-1, 1) # 전치행렬

                outputs_a = outputs.gather(1,a)
                pi_a = torch.exp(outputs_a) / exp_sum

                # pi_a = pi.gather(1,a)
                ratio = torch.exp(torch.log(pi_a) - torch.log(prob_a))  # a/b == exp(log(a)-log(b))

                surr1 = ratio * advantage_lst
                surr2 = torch.clamp(ratio, 1-eps_clip, 1+eps_clip) * advantage_lst
                pi_loss = - torch.min(surr1, surr2) - entropy_weight * current_entropy


                self.optimizer.zero_grad()
                pi_loss.mean().backward()
                self.optimizer.step()

    def train_net_history(self):
        self = self.cuda()
        # pre_advantage = 0.0
        for i in range(history_learning_time):
            torch.cuda.empty_cache()
            network_batch, job_waiting, a, r, next_network_batch, next_job_waiting, prob_a, entropy, sojourn_time, action_mask = self.make_batch_history()

            
            network_batch_1 = network_batch[0].clone().cuda()
            network_batch_2 = network_batch[1].clone().cuda()

            network_batch = [network_batch_1, network_batch_2]

            job_waiting_1 = job_waiting[0].clone().cuda()
            job_waiting_2 = job_waiting[1].clone().cuda()

            job_waiting = [job_waiting_1, job_waiting_2]

            a = a.clone().cuda()
            r = r.clone().cuda()

            next_network_batch_1 = next_network_batch[0].clone().cuda()
            next_network_batch_2 = next_network_batch[1].clone().cuda()

            next_network_batch = [next_network_batch_1, next_network_batch_2]

            next_job_waiting_1 = next_job_waiting[0].clone().cuda()
            next_job_waiting_2 = next_job_waiting[1].clone().cuda()

            next_job_waiting = [next_job_waiting_1, next_job_waiting_2]

            prob_a = prob_a.clone().cuda()
            entropy = entropy.clone().cuda()
            sojourn_time = sojourn_time.clone().cuda()
            gamma_gpu = torch.Tensor([gamma]).clone().cuda()
            action_mask = action_mask.clone().cuda()


            with torch.autograd.set_detect_anomaly(True):
                
                td_target = r + (gamma_gpu**sojourn_time) * self.v([next_network_batch, next_job_waiting])
                # print("r : ", r)
                # print("td_target : ", td_target)
                # print("v : ", self.v([network_batch, job_waiting]))
                # delta = td_target - self.v([network_batch, job_waiting])
                """
                
                advantage_lst = []
                advantage = pre_advantage
                for i, delta_t in enumerate(delta):
                    advantage = (gamma_gpu**sojourn_time[i][0]) * lmbda * advantage + delta_t[0]
                    advantage_lst.append([advantage])
                advantage_lst.reverse()
                advantage_lst = torch.tensor(advantage_lst, dtype=torch.float).cuda()

                pre_advantage = advantage
                """

                v_loss = val_loss_coef * F.smooth_l1_loss(self.v([network_batch, job_waiting]) , td_target.detach())
                #print("v_loss", v_loss)
                
                self.optimizer.zero_grad()
                v_loss.mean().backward()
                self.optimizer.step()

                td_target = r + (gamma_gpu**sojourn_time) * self.v([next_network_batch, next_job_waiting])
                delta = td_target - self.v([network_batch, job_waiting])
                # print(delta)

                pi, current_entropy, outputs = self.pi([network_batch, job_waiting])

                # true가 valid action

                outputs = outputs * action_mask

                exp_sum = torch.sum(torch.exp(outputs), dim=1) - torch.sum(action_mask.logical_not(), dim=1) # 0인 애들 빼줌
                exp_sum = exp_sum.view(-1, 1) # 전치행렬

                outputs_a = outputs.gather(1,a)
                pi_a = torch.exp(pi) / exp_sum

                pi_a = pi.gather(1,a)
                ratio = torch.exp(torch.log(pi_a) - torch.log(prob_a))  # a/b == exp(log(a)-log(b))

                surr1 = ratio * delta.detach()
                surr2 = torch.clamp(ratio, 1-eps_clip, 1+eps_clip) * delta.detach()
                pi_loss = - torch.min(surr1, surr2) - entropy_weight * current_entropy


                self.optimizer.zero_grad()
                pi_loss.mean().backward()
                self.optimizer.step()
                
                    


In [8]:
def main():
    global adjacency, entropy_weight, learning_rate
    model = actor_network()
    # model.load_state_dict(torch.load("./history30/model.pth")) # -1 , +1
    # model.load_state_dict(torch.load("./history0/model.pth")) # original
    reward_history = []
    v_history = []
    
    # network topology의 edges(GNN 예제 링크 참고)

    adjacency = torch.tensor(adjacency, dtype=torch.long)
    

    # node_state, link_state = env.get_state()  # 환경으로부터 실제 state를 관측해 옴(OMNeT++에서 얻어온 statistic로 대체되어야 함).
    # node_state = torch.tensor(node_state, dtype=torch.float)
    # link_state = torch.tensor(link_state, dtype=torch.float)
    # job_waiting_state = env.get_job_waiting_vector()

    # network_state = Data(x=node_state, edge_attr=link_state, edge_index=adjacency)

    # for each time step
    step = 1
    episode = 1

    max_reward = 0

    average_reward = 0
    average_reward_num = 0

    temp_history = []

    isStop = False
    node_selected_num = [0 for i in range(nodeNum)]
    void_selected_num = 0

    pre_state_1 = {}
    pre_state_2 = {}
    
    while True:
        model = model.to('cpu')
        msg = getOmnetMessage()
        
        if msg == "action": # omnet의 메세지, state 받으면 됨
            sendOmnetMessage("ok")
            state_1 = getOmnetMessage()
            state_2 = getOmnetMessage()

            if len(state_1) == 0:
                state_1 = state_2
            

            if len(pre_state_1) == 0: # action 시작
                state_1 = json.loads(state_1) # state 받았으므로 action 하면됨.
                state_2 = json.loads(state_2) # state 받았으므로 action 하면됨.
                
            else:
                state_1 = json.loads(state_1)
                pre_state_1['jobWaiting'] = state_1['jobWaiting']
                pre_state_1['sojournTime'] = state_1['sojournTime']
                state_1 = pre_state_1

                state_2 = json.loads(state_2)
                pre_state_2['jobWaiting'] = state_2['jobWaiting']
                pre_state_2['sojournTime'] = state_2['sojournTime']
                state_2 = pre_state_2
            
            sendOmnetMessage("ok") # 답장

            node_waiting_state_1 = np.array(eval(str(state_1['nodeState'])))
            node_processing_state_1 = np.array(eval(state_1['nodeProcessing']))
            link_state_1 = np.array(eval(state_1['linkWaiting']))
            job_waiting_state_1 = np.array(eval(state_1['jobWaiting']))
            activated_job_list_1 = eval(state_1['activatedJobList'])
            isAction_1 = int(state_1['isAction'])
            reward_1 = float(state_1['reward'])
            averageLatency_1 = float(state_1['averageLatency'])
            completeJobNum_1 = int(state_1['completeJobNum'])
            sojournTime_1 = float(state_1['sojournTime'])

            node_waiting_state_2 = np.array(eval(str(state_2['nodeState'])))
            node_processing_state_2 = np.array(eval(state_2['nodeProcessing']))
            link_state_2 = np.array(eval(state_2['linkWaiting']))
            job_waiting_state_2 = np.array(eval(state_2['jobWaiting']))
            activated_job_list_2 = eval(state_2['activatedJobList'])
            isAction_2 = int(state_2['isAction'])
            reward_2 = float(state_2['reward'])
            averageLatency_2 = float(state_2['averageLatency'])
            completeJobNum_2 = int(state_2['completeJobNum'])
            sojournTime_2 = float(state_2['sojournTime'])
            

        
            if averageLatency_2 != -1:
                writer.add_scalar("averageLatency/train", averageLatency_2 ,step)

            writer.add_scalar("completeJobNum/train", completeJobNum_2 ,step)
            writer.add_scalar("Reward/train", reward_2, step)

            # 이 timestep에서 발생한 모든 샘플에 똑같은 보상 적용.
            reward_2 = completeJobNum_2
            first_sample = True
            if is_train and len(pre_state_1) == 0:
                if len(temp_history) > 0:
                    temp_history[-1][8] = sojournTime_2
                for history in temp_history:
                    history[3] = reward_2
                    model.history.append(history)
                    model.put_data(history)

            model.history = model.history[-replay_buffer_size:]
            pre_state_1 = state_1
            pre_state_2 = state_2

            temp_history = []

            job_index = int(state_2['jobIndex'])

            #print('sojourn time :', sojournTime)


            node_state_1 = np.concatenate((node_waiting_state_1,node_processing_state_1) ,axis = 1)
            node_state_1 = torch.tensor(node_state_1, dtype=torch.float)

            node_state_2 = np.concatenate((node_waiting_state_2,node_processing_state_2) ,axis = 1)
            node_state_2 = torch.tensor(node_state_2, dtype=torch.float)

            #print(reward)

            link_state_1 = torch.tensor(link_state_1, dtype=torch.float)
            link_state_2 = torch.tensor(link_state_2, dtype=torch.float)

            job_waiting_num = 0
            job_waiting_queue = collections.deque()
            for job in job_waiting_state_2:
                if any(job): # 하나라도 0이 아닌 것 이 있으면 job이 있는것임.
                    job_waiting_num += 1
                    job_waiting_queue.append(job)
            
            job_waiting_state_1 = torch.tensor(job_waiting_state_1, dtype=torch.float).view(1, -1)
            job_waiting_state_2 = torch.tensor(job_waiting_state_2, dtype=torch.float).view(1, -1)
            # print(job_waiting_state)

            network_state_1 = Data(x=node_state_1, edge_attr=link_state_1, edge_index=adjacency)
            network_state_2 = Data(x=node_state_2, edge_attr=link_state_2, edge_index=adjacency)

            network_state = [network_state_1, network_state_2]
            job_waiting_state = [job_waiting_state_1, job_waiting_state_2]
            
            if average_reward_num == 0:
                average_reward = reward_2
                average_reward_num = 1
            else:
                average_reward = average_reward + (reward_2 - average_reward)/(average_reward_num + 1)
                average_reward_num += 1
                
            if step > 1:
                for i in range(nodeNum):
                    node_tag = "node/" + str(i) + "/train"
                    writer.add_scalar(node_tag, node_selected_num[i], step)

                writer.add_scalar("node/void/train", void_selected_num, step)
                    
                node_selected_num = [0 for i in range(nodeNum)] # node selected num 초기화
                void_selected_num = 0

                if reward_2 != 0:
                    with torch.no_grad():
                        writer.add_scalar("Value/train", model.v([network_state, job_waiting_state])[0], step)
                

                writer.flush()

            
            
            # print(job_waiting_queue)
            if job_waiting_num == 0:
                isAction_2 = False
                

            if isAction_2:
                job_idx = job_index
                job = job_waiting_queue.popleft()
                src = -1
                dst = 1
                for i in range(nodeNum):
                    if job[i] == -1:
                        src = i
                    if job[i] == 1:
                        dst = i

                if src == -1:
                    src = dst
                    
                #print(f"src : {src}, dst : {dst}")
                #print(job)
                subtasks = job[nodeNum:]
                offloading_vector = []
                temp_data = []
                scheduling_start = False
                # print(subtasks)
                step += 1
                #print("action start.")
                for order in range(len(subtasks)):
                    if subtasks[order] == 0:
                        break

                    network_state_1 = Data(x=node_state_1, edge_attr=link_state_1, edge_index=adjacency)
                    network_state_2 = Data(x=node_state_2, edge_attr=link_state_2, edge_index=adjacency)
                    network_state = [network_state_1, network_state_2]
                    job_waiting_state = [job_waiting_state_1, job_waiting_state_2]
                    # print(node_state.shape)
                    # print(link_state.shape)
                    # print(adjacency.shape)
                    # print(job_waiting_state.shape)
                    with torch.no_grad():
                        prob, entropy, output = model.pi([network_state, job_waiting_state])

                    writer.add_scalar("Entropy/train", entropy, step)
                    #print(f'prob : {prob}')
                    
                    m = Categorical(prob) 
                    node = m.sample().item()
                    #print(f'node : {node}')
                    
                    # void action 실험용
                    # node = nodeNum 
                    
                    
                    # void action 뽑으면
                    if node == nodeNum and not scheduling_start: 
                        # print("void")
                        prob[0] = torch.Tensor([0] * nodeNum + [1.0])
                        action_mask = [int(not scheduling_start) if i == node else int(scheduling_start) for i in range(nodeNum + 1)]

                        temp_history.append([
                            [network_state[0], network_state[1]], 
                            [job_waiting_state[0], job_waiting_state[1]], 
                            node, 0, 
                            [network_state[0], network_state[1]], 
                            [job_waiting_state[0], job_waiting_state[1]],
                            prob[0][node].item(), 
                            entropy, 0, action_mask]
                        )

                        sendOmnetMessage("void")

                        #print("action finish.")
                        
                        if getOmnetMessage() == "ok":
                            void_selected_num += 1
                            
                        break

                    else:
                        scheduling_start = True

                    if scheduling_start:

                        prob = torch.Tensor([F.softmax(output[0][:nodeNum], dim=0).tolist()])
                        prob = torch.cat([prob[0], torch.tensor([0])]) # void action masking
                        prob = torch.Tensor([prob.tolist()]).cuda()



                        m = Categorical(prob[0])
                        node = m.sample().item()
                        

                        offloading_vector.append(node)

                        node_selected_num[node] += 1
                        
                        # state transition(환경으로부터 매번 state를 업데이트하는게 아닌, 현재 state를 기반으로 action에 해당하는 waiting만 더해줌).
                        # 아래의 구체적인 코드는 이해하시기 보단 OMNeT++에서 받아온 데이터로 새로 짜시는게 빠를 것 같습니다.
                        next_node_state_1 = node_state_2.clone().detach()
                        next_job_waiting_state_1 = job_waiting_state_2.clone().detach()
                        node_waiting_state_1 = node_waiting_state_2.copy()
                        
                        next_node_state_2 = node_state_2.clone().detach()
                        next_job_waiting_state_2 = job_waiting_state_2.clone().detach()
                        node_waiting_state_2 = node_waiting_state_2.copy()
                        # print(next_job_waiting_state)
                        
                        next_node_state_2[node][modelNum * job_idx + order] += (subtasks[order]/100) # 100으로 나누는 이유 : 이렇게 해야 액션이 한쪽 노드로 쏠리게끔 학습이 되는 것을 어느정도 방지할 수 있는 것을 확인.
                        node_waiting_state_2[node][modelNum * job_idx + order] += (subtasks[order]/100)

                        # 100으로 안나눠주면 너무 큰 값이 state에 추가되어서 inference시 가중치랑 곱해지면서 액션이 한쪽으로 확 쏠리는 걸로 예상됨.

                        next_network_state_1 = Data(x=next_node_state_1, edge_attr=link_state_1, edge_index=adjacency)
                        next_network_state_2 = Data(x=next_node_state_2, edge_attr=link_state_1, edge_index=adjacency)
                        # next_job_waiting_state_1[0][nodeNum + order] = 0
                        next_job_waiting_state_2[0][nodeNum + order] = 0

                        next_network_state = [next_network_state_1, next_network_state_2]
                        next_job_waiting_state = [next_job_waiting_state_1, next_job_waiting_state_2]

                        action_mask = [int(not scheduling_start) if i == nodeNum else int(scheduling_start) for i in range(nodeNum + 1)]

                        temp_history.append([
                            [network_state[0], network_state[1]], 
                            [job_waiting_state[0], job_waiting_state[1]], 
                            node, 0, 
                            [next_network_state[0], next_network_state[1]], 
                            [next_job_waiting_state[0], next_job_waiting_state[1]],
                            prob[0][node].item(), 
                            entropy, 0, action_mask]
                        )
                        # model.put_data([network_state, job_waiting_state, node, 0, next_network_state, next_job_waiting_state, prob[0][node].item(), entropy])
                        node_state_1 = next_node_state_1
                        node_state_2 = next_node_state_2

                        job_waiting_state_1 = next_job_waiting_state_1
                        job_waiting_state_2 = next_job_waiting_state_2


                        pre_state_1['nodeState'] = str(node_waiting_state_1.tolist())
                        pre_state_2['nodeState'] = str(node_waiting_state_2.tolist())

                if len(offloading_vector) != 0: # for문을 다 돌면 -> void action 안뽑으면
                    # print(offloading_vector)
                    msg = str(offloading_vector)
                    sendOmnetMessage(msg)
                    
                    #print("action finish.")
                    if(getOmnetMessage() == "ok"):
                        pass

        elif msg == "stop":
            
            sendOmnetMessage("ok")
            pre_state_1 = {}
            pre_state_2 = {}
            
        elif msg == "episode_finish":
            sendOmnetMessage("ok")

            episodic_reward = getOmnetMessage()
            episodic_reward = json.loads(episodic_reward)
            
            finish_num = float(episodic_reward['reward'])
            complete_num = int(episodic_reward['completNum'])

            normalized_finish_num = model.return_normalize_reward(finish_num)
            
            writer.add_scalar("EpisodicReward/train", finish_num, episode)
            writer.add_scalar("NormalizedEpisodicReward/train", normalized_finish_num, episode)
            writer.add_scalar("CompleteNum/train", complete_num, episode)

            episode += 1
            sendOmnetMessage("ok")

            # learning_rate *= 0.995

            if finish_num > max_reward:
                modelPathName = pathName + "/max_model.pth"
                torch.save(model.state_dict(), modelPathName)
                max_reward = finish_num

            writer.add_scalar("AverageReward/train", average_reward, step)
            average_reward = 0
            average_reward_num = 0

            # entropy_weight *= 0.99


            # if len(model.data) > 0 and step % train_cycle == train_quitient:
            #     tm = localtime(time.time())
            #     time_string = strftime('%Y-%m-%d %I:%M:%S %p', tm)
            #     print(f"[{time_string}] training....")
            #     model.train_net()
            #     modelPathName = pathName + "/model.pth"
            #     torch.save(model.state_dict(), modelPathName)
            #     writer.add_scalar("AverageReward/train", average_reward, step)
            #     average_reward = 0
            #     average_reward_num = 0
            #     tm = localtime(time.time())
            #     time_string = strftime('%Y-%m-%d %I:%M:%S %p', tm)
            #     print(f"[{time_string}] training complete")
            if is_train:

               
                
                tm = localtime(time.time())
                time_string = strftime('%Y-%m-%d %I:%M:%S %p', tm)
                print(f"[{time_string}] training....")
                model.train_net()
                tm = localtime(time.time())
                time_string = strftime('%Y-%m-%d %I:%M:%S %p', tm)
                print(f"[{time_string}] training complete")

                tm = localtime(time.time())
                time_string = strftime('%Y-%m-%d %I:%M:%S %p', tm)
                print(f"[{time_string}] training replay buffer....")
                model.train_net_history()
                tm = localtime(time.time())
                time_string = strftime('%Y-%m-%d %I:%M:%S %p', tm)
                print(f"[{time_string}] training complete")
                
                modelPathName = pathName + "/model.pth"
                torch.save(model.state_dict(), modelPathName)
                
                
                

        
if __name__ == '__main__':
    main()

[2022-09-07 05:39:12 PM] training....
[2022-09-07 05:39:15 PM] training complete
[2022-09-07 05:39:15 PM] training replay buffer....
[2022-09-07 05:39:24 PM] training complete
[2022-09-07 05:39:29 PM] training....
[2022-09-07 05:39:31 PM] training complete
[2022-09-07 05:39:31 PM] training replay buffer....
[2022-09-07 05:39:40 PM] training complete
[2022-09-07 05:39:44 PM] training....
[2022-09-07 05:39:46 PM] training complete
[2022-09-07 05:39:46 PM] training replay buffer....
[2022-09-07 05:39:56 PM] training complete
[2022-09-07 05:39:59 PM] training....
[2022-09-07 05:40:00 PM] training complete
[2022-09-07 05:40:00 PM] training replay buffer....
[2022-09-07 05:40:09 PM] training complete
[2022-09-07 05:40:14 PM] training....
[2022-09-07 05:40:16 PM] training complete
[2022-09-07 05:40:16 PM] training replay buffer....
[2022-09-07 05:40:25 PM] training complete
[2022-09-07 05:40:28 PM] training....
[2022-09-07 05:40:29 PM] training complete
[2022-09-07 05:40:29 PM] training repla

C:\Users\user\anaconda3\envs\omnetTest\lib\site-packages\torch\autograd\__init__.py:173: UserWarning: Error detected in SmoothL1LossBackward0. Traceback of forward call that caused the error:
  File "C:\Users\user\anaconda3\envs\omnetTest\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\user\anaconda3\envs\omnetTest\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "C:\Users\user\anaconda3\envs\omnetTest\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\user\anaconda3\envs\omnetTest\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
    app.start()
  File "C:\Users\user\anaconda3\envs\omnetTest\lib\site-packages\ipykernel\kernelapp.py", line 677, in start
    self.io_loop.start()
  File "C:\Users\user\anaconda3\envs\omnetTest\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
    self.asyncio_loop.

RuntimeError: Function 'SmoothL1LossBackward0' returned nan values in its 0th output.

In [ ]:
output = torch.Tensor([[-0.0256,  0.0438,  0.0015, -0.0308, -0.0386, -0.0370, -0.0170, -0.0413]])
print(output[0][:nodeNum])


In [ ]:
a = [[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [16.666667], [16.666667], [16.666667], [16.666667], [16.666667], [16.666667], [16.666667], [16.666667], [16.666667], [16.666667], [16.666667], [16.666667], [16.666667], [16.666667], [16.666667], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [17.941176], [17.941176], [17.941176], [17.941176], [17.941176], [43.333333], [43.333333], [43.333333], [43.333333], [55.333333], [55.333333], [55.333333], [55.333333], [55.333333], [55.333333], [55.333333], [55.333333], [55.333333], [10.0], [10.0], [10.0], [10.0], [10.0], [26.25], [26.25], [26.25], [26.25], [24.054054], [24.054054], [24.054054], [24.054054], [24.054054], [14.137931], [14.137931], [41.5], [41.5], [21.538462], [21.538462], [37.058824], [21.315789], [21.315789], [21.315789], [21.315789], [24.571429], [24.571429], [24.571429], [24.571429], [32.307692], [32.307692], [32.307692], [32.307692], [7.592593], [7.592593], [7.592593], [7.592593], [7.592593], [14.705882], [112.5], [21.666667], [21.666667], [21.666667], [21.666667], [21.666667], [19.574468], [19.574468], [19.574468], [19.574468], [19.574468], [68.0], [68.0], [68.0], [17.692308], [17.692308], [17.692308], [43.636364], [43.636364], [43.636364], [43.636364], [1.52381], [1.52381], [1.52381], [1.52381], [1.52381], [2.966102], [35.333333], [35.333333], [3.608247], [3.608247], [3.608247], [3.608247], [3.608247], [17.037037], [17.037037], [17.037037], [10.0], [10.0], [10.0], [34.074074], [34.074074], [34.074074], [34.074074], [10.0], [87.142857], [77.272727], [10.09901], [10.09901], [70.833333], [70.833333], [70.833333], [70.833333], [59.375], [59.375], [59.375], [59.375], [33.214286], [33.214286], [33.214286], [46.0], [46.0], [46.0], [46.0], [20.357143], [20.357143], [20.357143], [20.357143], [28.4375], [22.857143], [22.857143], [22.857143], [22.857143], [56.923077], [56.923077], [56.923077], [56.923077], [56.923077], [16.842105], [18.666667], [18.666667], [18.666667], [18.666667], [18.666667], [5.806452], [5.806452], [5.806452], [35.0], [35.0], [35.0], [35.0], [32.352941], [32.352941], [10.0], [10.0], [16.666667], [16.666667], [16.666667], [16.666667], [34.375], [34.375], [34.375], [34.375], [34.375], [35.172414], [35.172414], [35.172414], [14.0], [14.0], [14.0], [8.695652], [18.4], [16.153846], [16.153846], [16.153846], [16.153846], [16.153846], [27.575758], [27.575758], [14.761905], [14.761905], [14.761905], [22.1875], [22.1875], [59.333333], [10.0], [10.0], [10.0], [27.666667], [20.0], [20.0], [20.0], [118.0], [118.0], [118.0], [118.0], [10.0], [10.0], [19.677419], [19.677419], [19.677419], [19.677419], [40.769231], [40.769231], [40.769231], [19.393939], [19.393939], [19.393939], [19.393939], [7.173913], [7.173913], [7.173913], [31.052632], [31.052632], [10.0], [25.714286], [25.714286], [25.714286], [25.714286], [28.888889], [40.526316], [31.363636], [31.363636], [31.363636], [31.363636], [21.5], [21.5], [21.5], [92.857143], [92.857143], [92.857143], [8.148148], [10.0], [16.666667], [16.666667], [16.666667], [16.666667], [16.666667], [83.994253], [83.994253], [83.994253], [83.994253], [83.994253], [83.994253], [3.873874], [3.873874], [3.873874], [3.873874], [5.416667], [5.416667], [5.416667], [9.904762], [9.904762], [9.904762], [9.904762], [9.904762], [34.0], [26.956522], [26.956522], [26.956522], [3.714286], [3.714286], [3.714286], [3.714286], [7.037037], [7.037037], [7.037037], [7.037037], [24.615385], [16.25], [16.25], [27.142857], [27.142857], [4.736842], [4.736842], [4.736842], [10.0], [23.0], [23.0], [34.583333], [34.583333], [10.0], [10.0], [34.705882], [19.02439], [19.02439], [19.02439], [19.02439], [19.02439], [40.0], [40.0], [40.0], [44.444444], [44.444444], [44.444444], [44.444444], [44.444444], [20.416667], [20.416667], [20.416667], [5.30303], [5.30303], [5.30303], [6.666667], [6.666667]]

a = np.array(a)

print(sum(a))
print(np.std(a))
print(np.mean(a))
a = (a-np.mean(a)) / np.std(a)
a = a.tolist()
print(a)

In [ ]:
a = torch.Tensor([1, 2, 3, 4, 5])
b = torch.Tensor([[1], [2], [3], [4], [5]])

a = torch.Tensor([3])

print(a**b)